In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [28]:
train =pd.read_csv("train_play.csv",parse_dates=['time'])
train.head()

,row_id,time,x,y,direction,congestion
0,0,1991-04-01,0,0,EB,70
1,1,1991-04-01,0,0,NB,49
2,2,1991-04-01,0,0,SB,24
3,3,1991-04-01,0,1,EB,18
4,4,1991-04-01,0,1,NB,60


In [29]:
test =pd.read_csv("test_play.csv",parse_dates=['time'])
test.head()

,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB


In [25]:
train['month'] = train['time'].dt.month
train['day'] = train['time'].dt.day
train['hour'] = train['time'].dt.hour
train['weekday'] = train['time'].dt.weekday

test['month'] = test['time'].dt.month
test['day'] = test['time'].dt.day
test['hour'] = test['time'].dt.hour
test['weekday'] = test['time'].dt.weekday

In [26]:
le = LabelEncoder()

dir_en=pd.DataFrame(le.fit_transform(train['direction']))
X_train = train.drop(['time', 'congestion','direction'], axis=1)
X_train['direction']=dir_en

y_train = np.log1p(train['congestion'])

In [30]:
d_en=pd.DataFrame(le.fit_transform(test['direction']))

X_test =  test.drop(['time','row_id','direction'], axis=1)
X_test['direction'] = dir_en

print(X_train.columns)
print(X_test.columns)

Index(['x', 'y', 'month', 'day', 'hour', 'weekday', 'direction'], dtype='object')
Index(['x', 'y', 'direction'], dtype='object')


In [31]:
X_train['weekday'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [33]:
#Linear regression

lr = LinearRegression()
lr.fit(X_train,y_train)


kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {}
gcv = GridSearchCV(lr, param_grid = params, cv = kfold, scoring = 'neg_mean_absolute_error')
gcv.fit(X_train,y_train)
print(gcv.best_score_)
print(gcv.best_params_)

y_pred = np.expm1(gcv.predict(X_test))
y_pred

-0.30876171835430843
{}


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- day
- hour
- month
- weekday


In [14]:
submit = pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

KeyError: 'row_id'

In [15]:
#using elasticnet

el = ElasticNet(alpha=0.4, l1_ratio=0.2)
el.fit(X_train,y_train)

kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {}
gcv = GridSearchCV(el, param_grid = params, cv = kfold, scoring = 'neg_mean_absolute_error')
#gcv.fit(X_train,y_train)
#print(gcv.best_score_)
#print(gcv.best_params_)

y_pred = np.expm1(el.predict(X_test))
y_pred

NameError: name 'X_train' is not defined

In [16]:
submit=pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit


KeyError: 'row_id'

In [17]:
#using lightGBM

lgb=LGBMRegressor()

#lgb.fit()

#lgb.fit(X_train,y_train, eval_metric="mae")
kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {'learning_rate' : np.linspace(0.001, 0.9, 10), 'max_depth' : [None, 3, 4], 'n_estimators' : [25, 50, 100]}
gcv1 = GridSearchCV(lgb, param_grid = params, cv = kfold, scoring = 'neg_root_mean_squared_error')
gcv1.fit(X_train, y_train)
print(gcv1.best_score_)
print(gcv1.best_params_)

y_pred=np.expm1(gcv1.predict(X_test))
y_pred

NameError: name 'X_train' is not defined

In [18]:
submit=pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

KeyError: 'row_id'

In [19]:
#using xgb
xgb = XGBRegressor(random_state=24)
xgb.fit(X_train,y_train)

kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {'learning_rate' : np.linspace(0.001, 0.9, 10), 'max_depth' : [None, 3, 4], 'n_estimators' : [25, 50, 100]}
gcv1 = GridSearchCV(xgb, param_grid = params, cv = kfold, scoring = 'neg_root_mean_squared_error')
gcv1.fit(X_train, y_train)
print(gcv1.best_score_)
print(gcv1.best_params_)

y_pred=np.expm1(gcv1.predict(X_test))
y_pred

NameError: name 'X_train' is not defined

In [20]:
#using cat boost

cat = CatBoostRegressor(random_state=24)
cat.fit(X_train, y_train)

y_pred=np.expm1(cat.predict(X_test))

NameError: name 'X_train' is not defined

In [21]:
submit=pd.DataFrame({'row_id':test['row_id'],'congestion':y_pred})
submit

KeyError: 'row_id'